# Predicción de Precios de Coches Usados - Modelo de Machine Learning

## Análisis y procesamiento de datos

### Introduccion
La estimación del precio de un coche de segunda mano es un problema complejo, ya que depende de múltiples factores como el año, el kilometraje, la marca y otras características del vehículo. Para abordar esta tarea, se ha recopilado un conjunto de datos con ventas históricas de coches usados, incluyendo el precio real de venta. A partir de esta información, se aplicarán técnicas de Machine Learning para analizar y procesar los datos, entrenar modelos predictivos y evaluar su desempeño, con el objetivo de sugerir precios en portales de compraventa online o realizar tasaciones automáticas.

### Objetivo
Desarrollar e implementar un modelo de aprendizaje supervisado capaz de predecir el precio de venta de coches de segunda mano, evaluando su desempeño mediante el error absoluto medio (MAE) en el conjunto de validación, considerando satisfactorio un MAE inferior a 3,000 €, con el fin de generar predicciones precisas útiles para la tasación automática o la sugerencia de precios en portales de compraventa online.

In [184]:
# Importamos las librerias necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing as prep
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error , mean_absolute_percentage_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import precision_score , recall_score , confusion_matrix
from sklearn.metrics import roc_curve , auc
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.tree import plot_tree , export_text
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
#HIperparametros
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from joblib import dump






### Cargar datos
En este proyecto utilizaremos Pandas y NumPy para la manipulación y análisis de datos, Matplotlib y Seaborn para la visualización gráfica, y Scikit-Learn para el desarrollo y evaluación de los modelos de machine learning.

In [185]:
train_data = pd.read_csv("data/dataset_coches_train.csv")
test_data= pd.read_csv("data/dataset_coches_test.csv")

### Explorar datos iniciales
1. Informacion General de los datos

En primer lugar a ver qué información contiene el conjunto de datos : el tamaño,filas y columnas , si hay registros duplicados o ausentes, etc.

In [195]:

print("============== Primeras 5 filas ================")
print(train_data.head()) 
print("============= Últimas 5 filas ====================")
print(train_data.tail())

============== Primeras 5 filas ================
      ID     marca    modelo  fecha tipo_cambio  total_km tipo_combustible  \
0  13207    hyundi  Santa Fe   2019   Semi-Auto      4223           Diesel   
1  17314  vauxhall       GTC   2015      Manual     47870           Diesel   
2  12342      audi       RS4   2019   Automatic      5151           Petrol   
3  13426        vw  Scirocco   2016   Automatic     20423           Diesel   
4  16004     skoda     Scala   2020   Semi-Auto      3569           Petrol   

   consumo  tipo_motor   tasa  precio  
0     5.91         2.2  145.0   31995  
1     3.91         2.0  125.0    7700  
2     8.08         2.9  145.0   58990  
3     4.08         2.0   30.0   12999  
4     4.99         1.0  145.0   16990  
============= Últimas 5 filas ====================
         ID marca    modelo  fecha tipo_cambio  total_km tipo_combustible  \
4955  16898  merc  GL Class   2015   Automatic     24314           Diesel   
4956  14416   bmw  6 Series   2017   

In [ ]:
print("======== Informacion general del Dataframe =========")
train_data.info()



In [ ]:
print("=========== Resumen estadistico de las variables ========")
train_data.describe(include="all")


In [187]:
train_data.isnull().sum()


ID                    0
marca                 0
modelo                0
fecha                 0
tipo_cambio         148
total_km              0
tipo_combustible      0
consumo             396
tipo_motor            0
tasa                  0
precio                0
dtype: int64

In [ ]:
print(f'Número de valores duplicados: {train_data.duplicated().sum()}')

El dataset contiene 4960 registros , distribuidos con 11 columnas con sus respectivas caracteristicas , siete de tipo numerico y cuatro de tipo categorico.En un análisis inicial se identifican 9 marcas y 90 modelos distintos, siendo Mercedes-Benz la marca más frecuente.
La antigüedad media de los vehículos corresponde al año 2017, lo que indica que, en general, se trata de coches relativamente recientes.
Por una lado la variable objetivo precio, los valores oscilan entre 450 € y 145.000 €, lo que sugiere la presencia de outliers significativos que deberán analizarse o tratarse antes del modelado.Por otro lado A partir del metodo nulos y duplicados  se identificó que la variable tipo_cambio presenta 148 valores faltantes, mientras que la variable consumo contiene 396 valores faltantes, lo que evidencia la necesidad de aplicar técnicas de imputación o eliminación antes del entrenamiento del modelo y por ultimo no se encontro valores nulos.

### Visualizacion de datos

 Vamos a explorar las características categóricas y numéricas para observar la distribución de nuestros datos.


In [ ]:
cat_cols = train_data.select_dtypes(include='object').columns
num_cols = train_data.select_dtypes(include=['int64', 'float64']).columns

# Revisamos los valores únicos de cada columna con el método nunique
print("\n=== VALORES ÚNICOS POR COLUMNA ===")
for col in train_data.columns:
    unique_vals = train_data[col].nunique()
    print(f"{col}: {unique_vals} valores únicos")
    if unique_vals < 20:
        print(f"  Valores: {train_data[col].unique()}")


In [ ]:
for col in num_cols:
    plt.figure(figsize=(6, 4))
    plt.hist(
        train_data[col],
        bins=20,
        edgecolor='black', 
        alpha=0.7         
    )

    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()

Se analizaron las distribuciones de las variables numéricas mediante histogramas, lo que permitió observar la forma de los datos, detectar posibles asimetrías y valores atípicos. Algunas variables presentan distribuciones sesgadas, lo cual es común en datos de mercado de vehículos usados.

Para finalizar el análisis de estos datos, comprobemos la correlación entre la variable objetivo y las variables numéricas.

In [181]:

train_data[num_cols].corr()['precio'].sort_values(ascending=False)




precio        1.000000
tipo_motor    0.624899
fecha         0.501296
consumo       0.261857
tasa          0.144652
ID            0.012129
total_km     -0.478705
Name: precio, dtype: float64

El análisis de correlación muestra que el precio del vehículo presenta una relación positiva moderada con el año de fabricación y una relación negativa significativa con el kilometraje, lo cual concuerda con la lógica del mercado de vehículos usados. Asimismo, se observa una alta correlación positiva con el tipo de motor, aunque esta variable requiere un tratamiento categórico adecuado antes de su uso en el modelo. Por otro lado las variables como la tasa y consumo presentan una relacion debil lo cual no se tendran en cuenta.

In [ ]:
# comprobemos la correlación entre la variable objetivo y las variables categoricas como la marca 
train_data.groupby('marca')['precio'].mean().sort_values(ascending=False) 

train_data.boxplot(column='precio', by='marca', rot=70 )
sns.boxplot(x='marca', y='precio', hue='marca', data=train_data, palette='Set3', legend=False) 
plt.title('Precio por marca') 
plt.suptitle('')
plt.show()

Se observa claramente que las marcas de autos determinan en gran medida el precio.Marcas premium como Audi y Mercedes tienen medianas de precio mucho más altas (alrededor de 80.000–100.000), mientras que marcas como Skoda, Vauxhall, Hyundai y Ford tienen medianas mucho más bajas (aproximadamente 10.000–40.000).Por otro lado se presentan outliers podrían corresponder a modelos de lujo, ediciones especiales o autos importados que son mucho más caros que los modelos típicos de la marca.

## Procesamiento del dataset

### Imputacion de valores nulos

In [ ]:
print(train_data[cat_cols.tolist() + ['precio']].isna().sum())
# Crear un dataframe limpio solo con las columnas que vamos a analizar
""" Opción A: Eliminar filas con NaN (más simple y segura)
train_data_clean = train_data.dropna(subset=cat_cols.tolist() + ['precio']) """

""" # Rellenar con la categoría más frecuente
for col in cat_cols:
    train_data[col] = train_data[col].fillna(train_data[col].mode()[0]) """

""" from scipy.stats import f_oneway

eta_sq_dict = {}
for col in cat_cols:
    groups = [train_data_clean['precio'][train_data_clean[col]==cat] for cat in train_data_clean[col].unique()]
    f_stat, p_val = f_oneway(*groups)
    n_total = len(train_data_clean)
    eta_sq = f_stat * (len(groups[0])-1) / (f_stat * (len(groups[0])-1) + n_total - len(groups))
    eta_sq_dict[col] = eta_sq

# Ordenar y mostrar
eta_sq_series = pd.Series(eta_sq_dict).sort_values(ascending=False)
print(eta_sq_series) """

In [201]:
# Imputación Educational_level con el valor más común
top_value = train_data.tipo_cambio.value_counts().index[0]
train_data["tipo_cambio"] = train_data["tipo_cambio"].fillna(top_value)
train_data["tipo_cambio"].isnull().sum()

np.int64(0)

In [ ]:
# Imputación consumo con la media
mean_value = train_data.consumo.mean()
train_data["consumo"] = train_data["consumo"].fillna(mean_value)
train_data["consumo"].isnull().sum()

np.int64(0)